# **Agent Tools & Handoffs**

# **What are Tools in Agents?**

In Agentic AI:

* **LLMs can’t actually “do” things** on their own (like fetch real data, run code, or call APIs).
* **Tools** are how you let an agent *extend its powers* by calling **functions you define in Python** (or external APIs).
* The agent decides *when* and *how* to call a tool, based on the user’s input.

Think of tools as:
👉 *“skills or superpowers that an agent can use when it can’t just rely on text reasoning.”*

<br>

---

<br>

# **How Tools Work in the Agent Loop**

1. **User gives input** → “What’s the weather in Karachi?”
2. Agent tries to solve it.
3. If the model realizes it needs external info, it says:

   > *“I should call the `get_weather` tool with city=Karachi.”*
4. The SDK executes your Python function (`get_weather("Karachi")`).
5. The tool’s output is fed back to the agent.
6. The agent produces the **final answer** to the user.

<br>

---

<br>

# **Defining Tools**

The SDK makes this **super simple**. You use the `@function_tool` decorator.

Example:

```python
from agents import function_tool

@function_tool
def get_weather(city: str) -> str:
    """Fetches weather for a given city."""
    return f"The weather in {city} is sunny with 30°C."
```

* The decorator does the heavy lifting:

  * It tells the agent **what the function does** (docstring).
  * It registers the function signature (`city: str`).
  * It makes the tool available for the agent’s reasoning loop.

<br>

---

<br>

# **Adding Tools to an Agent**

When you create the agent, just pass a list of tools:

```python
from agents import Agent

agent = Agent(
    name="WeatherBot",
    instructions="You answer weather-related questions.",
    tools=[get_weather],   # ✅ now agent can call this tool
)
```

<br>

---

<br>

# **Why Tools are Powerful**

* **APIs** → connect to weather, stock prices, databases.
* **Math/logic** → do calculations the LLM can’t.
* **Custom workflows** → trigger emails, run SQL queries, call other services.
* **Multi-agent collaboration** → one agent can call another as a “tool.”

---

✅ **In short:**
Tools let you go *beyond text*.
They turn an LLM from a *talker* → into a *doer*.



In [ ]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.7 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
import os

from agents import  Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, function_tool, set_tracing_disabled
from google.colab import userdata

# Debug Logging
# from agents import enable_verbose_stdout_logging,
# enable_verbose_stdout_logging()

# Disable tracing globally
set_tracing_disabled(disabled=True)

# Setup key
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

MODEL= "gemini-2.5-flash"

external_client= AsyncOpenAI(
    api_key= GEMINI_API_KEY,
    base_url= "https://generativelanguage.googleapis.com/v1beta/openai/"
)

model= OpenAIChatCompletionsModel(
    model= MODEL,
    openai_client= external_client,
)

# Example tool
@function_tool
def get_weather(city: str) -> str:
  print("[DEBUG] getting the weather data")
  return f"The weather in {city} is sunny."

# Agent
agent = Agent(
    name= "Assistant",
    instructions= "You only respond in haikus.",
    model= model,
    tools= [get_weather],
)

async def main():
  result = await Runner.run(starting_agent= agent, input="What is current weather in Karachi.")
  print(result.final_output)

if __name__ == "__main__":
  asyncio.run(main())

[DEBUG] getting the weather data
Sun shines bright today,
In Karachi's warm embrace,
Clear skies, no clouds roam.


# **What is a Handoff?**

A **handoff** happens when one agent decides it **can’t (or shouldn’t) finish the task itself** and instead passes control to another agent.

Think of it like a customer support team:

* The **triage agent** (front desk) takes your request.
* If it’s a billing issue, it **hands off** to the billing agent.
* If it’s technical, it **hands off** to the tech agent.

👉 In agent systems, a **handoff = structured baton pass** between agents.

<br>

---

<br>

# **How It Works in the Agent Loop**

1. **Agent A receives input**
   → Example: “Summarize this PDF.”

2. **Agent A realizes**:

   > “I don’t handle documents. I should hand this off to `DocumentAgent`.”

3. **Handoff step is emitted** by Agent A.

   * The SDK pauses execution.
   * The baton is passed to the target agent.

4. **Agent B takes over** with the same context/input.

   * Processes the task.
   * Produces its own output.

5. **Control returns to the loop** (depending on configuration).

<br>

---

<br>

# **Why Handoff is Useful**

* **Multi-Agent Specialization**: Agents can each be good at one thing (math, research, coding, support).
* **Cleaner logic**: Instead of cramming one giant agent with all tools, you route to the right expert.
* **Triage agents** use handoff heavily → they *decide where to send the request*.

<br>

---

<br>

# **Difference Between Tool Calls and Handoffs**

* **Tool** = agent says: *“Let me call this helper function and use its result.”*
* **Handoff** = agent says: *“I can’t do this; another whole agent should take over.”*

<br>

---

<br>

✅ **In short:**
Handoff is how agents **pass control to another agent** in the loop.
It’s the backbone of **multi-agent systems** where each agent is a specialist.



In [ ]:
!pip install -Uq openai-agents  "openai-agents[litellm]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 56.9 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio

from agents import Agent, Runner, set_tracing_disabled, function_tool
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

set_tracing_disabled(disabled= True)

MODEL= "gemini/gemini-2.5-flash"
GEMINI_API_KEY= userdata.get("GEMINI_API_KEY")

@function_tool
def get_weather(city: str) -> str:
  return f"The weather in {city} is sunny."

weather_agent = Agent(
    name='WeatherAssistant',
    instructions= "You will answer weather relevent questions",
    model= LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
    tools= [get_weather],
    handoff_description= "Weather Assistant is a specialized for all weather queries.",
)

panaversity_agent= Agent(
    name= "PanaversityAssistant",
    instructions= "You will answer Panaversity related queries",
    model= LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
    handoff_description="Panaversity Assistant is a specializes for all panaversity releated queries. Panaversity is an educational program."
)

triage_agent = Agent(
    name= "GeneralAssistant",
    instructions= "You will chat with user for general questions and handoff to Specialized agent",
    model= LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
    handoffs= [weather_agent, panaversity_agent],
    tools= [get_weather]
)

result = Runner.run_sync(triage_agent, "Hello")
print(result.final_output)

Hello! How can I help you today?



In [ ]:
dir(result)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_last_agent',
 'context_wrapper',
 'final_output',
 'final_output_as',
 'input',
 'input_guardrail_results',
 'last_agent',
 'last_response_id',
 'new_items',
 'output_guardrail_results',
 'raw_responses',
 'to_input_list']

In [ ]:
result.last_agent.name

'GeneralAssistant'

In [ ]:
result= Runner.run_sync(triage_agent, "What is the weather in karachi")
print(result.final_output)
print(result.last_agent.name)

print("-" * 10)

result= Runner.run_sync(triage_agent, "What is panaversity")
print(result.final_output)
print(result.last_agent.name)

print("-" * 10)

result= Runner.run_sync(triage_agent, "Hi There!")
print(result.final_output)
print(result.last_agent.name)

[DEBUG] getting the weather data
The weather in Karachi is sunny.
WeatherAssistant
----------
Panaversity is an innovative educational platform designed to offer a comprehensive and universal learning experience. It combines the concept of a "university" (indicating a focus on higher learning and knowledge acquisition) with the prefix "Pana-" (meaning "all," "every," or "worldwide").

Therefore, Panaversity represents a broad-reaching educational initiative, potentially offering diverse courses, programs, and resources accessible to a global audience, aiming to provide holistic and inclusive learning opportunities across various fields of study.
PanaversityAssistant
----------
Hello! How can I help you today?

GeneralAssistant


In [ ]:
result= Runner.run_sync(triage_agent, "what is panaversity and also what is the weather of kaarchi today?")
print(result.final_output)
print(result.last_agent.name)

The weather in Karachi is sunny.

Panaversity is an educational program. Would you like to know more about it?
GeneralAssistant
